## Modelo Supervisionado 

**Projeto**: Expansão por grupos — classificador de “Bom Desempenho” por região×grupo
**Objetivo**: Treinar e avaliar um RandomForest com pipeline (pré-processamento + OHE), escolher limiar ótimo por F1, salvar o modelo e ranquear candidatos.

### Configuração inicial, importações e parâmetros globais

- Importa bibliotecas essenciais para:
  - Manipulação e análise de dados (`pandas`, `numpy`)
  - Visualização (`matplotlib`)
  - Modelagem supervisionada (`scikit-learn`)
  - Persistência de modelos (`joblib`)

- Define configurações globais:
  - Supressão de warnings não críticos
  - Ajustes de exibição do pandas (linhas, colunas e largura)

- Estabelece parâmetros do projeto:
  - Caminhos para datasets e artefatos
  - Hiperparâmetros globais (ex.: `TOP_N`, `TOP_K`, `RANDOM_STATE`)

- Implementa função `read_csv_flex`:
  - Busca arquivos `.csv` em múltiplos diretórios possíveis
  - Testa diferentes separadores (`;`, `,`, autodetecção)
  - Retorna `DataFrame` válido ou lança erro com histórico de tentativas


In [ ]:
# ===== Imports principais =====
import glob
from pathlib import Path
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ===== Módulos do scikit-learn =====
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit  # validação estratificada
from sklearn.compose import ColumnTransformer                               # pré-processamento por tipo de coluna
from sklearn.pipeline import Pipeline                                       # pipeline unificado
from sklearn.impute import SimpleImputer                                    # imputação de valores faltantes
from sklearn.preprocessing import OneHotEncoder                             # codificação categórica
from sklearn.ensemble import RandomForestClassifier                         # classificador de árvores
from sklearn.metrics import (classification_report, confusion_matrix, roc_auc_score,
                             average_precision_score, precision_recall_curve, roc_curve)  # métricas de avaliação
from sklearn.inspection import permutation_importance                       # interpretabilidade
from joblib import dump, load                                               # persistência de modelos

# ===== Configurações globais =====
warnings.filterwarnings("ignore", category=UserWarning)  # suprime avisos não críticos
pd.set_option("display.max_columns", None)              # exibe todas as colunas
pd.set_option("display.width", 220)                     # define largura máxima de exibição
pd.set_option("display.max_rows", 200)                  # aumenta limite de linhas exibidas

# ===== Parâmetros do projeto =====
CAMINHO_DF_HIST = "../../../database/dataset gerado/dataset_limpo.csv"                 # dataset histórico (limpo)
ARQ_MODELO      = Path("../../../database/dataset gerado/modelo_grupo_chilli_simplificado.joblib")    # arquivo para salvar/carregar modelo
ARQ_NOVAS       = "../../../database/dataset gerado/sugestoes_expansao_para_supervisionado"  # sugestões do não supervisionado
TOP_N           = 10                                                 # número de categorias no ranking final
TOP_K           = 1                                                  # top-k localidades
FORCAR_TREINO   = False                                              # se True, força re-treinamento
LIMIAR          = None                                               # limiar de decisão (se não definido, será calibrado)
RANDOM_STATE    = 42                                                 # semente aleatória para reprodutibilidade

# ===== Função utilitária para leitura flexível de CSV =====
def read_csv_flex(base_path_or_stem: str) -> pd.DataFrame:
    """
    Lê um arquivo CSV de forma flexível, testando múltiplos caminhos e separadores.
    Retorna um DataFrame se encontrado, caso contrário levanta FileNotFoundError.
    """
    # Constrói lista de candidatos (caminho direto, com extensão, em pastas comuns)
    candidates = [
        Path(base_path_or_stem),
        Path(f"{base_path_or_stem}.csv"),
        Path("data") / f"{base_path_or_stem}.csv",
        Path("datasets") / f"{base_path_or_stem}.csv",
    ]
    # Acrescenta arquivos encontrados no diretório de forma recursiva
    for p in glob.glob(f"**/{Path(base_path_or_stem).stem}*.csv", recursive=True):
        candidates.append(Path(p))

    tried = []  # registra tentativas de leitura
    for path in candidates:
        if not path.exists():
            tried.append(str(path))
            continue
        # Testa diferentes separadores
        for sep in [None, ';', ',']:
            try:
                df = pd.read_csv(path, sep=sep, engine="python")
                print(f"✔ Arquivo carregado de: {path}")
                return df
            except Exception:
                tried.append(f"{path}(sep={sep})")
    # Se nenhum arquivo foi lido, lança erro com histórico de tentativas
    raise FileNotFoundError("CSV não encontrado. Tentativas: " + " | ".join(tried))


### Construção da base supervisionada (features + rótulo por REGIÃO × GRUPO)

- **Finalidade:** consolidar, em uma linha por par `REGIÃO × GRUPO`, as variáveis explicativas e o rótulo binário de desempenho.
- **Entradas principais:** `df_hist` com colunas de preço (bruto e líquido), desconto, identificador de loja, região da loja e grupo de produto.
- **Regras de negócio:**
  - O rótulo `Bom_Desempenho` marca como 1 os **Top-K grupos por região** (maior receita líquida por par `REGIÃO × GRUPO`); demais pares recebem 0.
  - As **features** incluem: receita total por região, número de lojas por região, receita por loja na região e **medianas por par `REGIÃO × GRUPO`** (preço de varejo e desconto).
- **Artefatos auxiliares:**
  - `catalogo` agrega metadados por grupo (ex.: `Grupo_Produto` e mediana de preço de varejo) para cruzar depois com as novas localidades.
  - `agg_reg` e `grp_meds` ficam disponíveis para análises e enriquecimentos adicionais.
- **Validações rápidas:**
  - Conferir presença das colunas exigidas antes de agrupar.
  - Conferir distribuição de `Bom_Desempenho` e a forma de `X_all`/`y_all`.
  - Visualizar distribuição do alvo com um gráfico de barras simples.


In [ ]:
# FEATURES + RÓTULO (1 linha por REGIÃO×GRUPO) 
def build_train_table(df_hist: pd.DataFrame, top_k: int = 1):
    # Checagem de colunas necessárias para garantir integridade mínima da base
    need = ["ID_Loja","Desconto","Total_Preco_Varejo","Total_Preco_Liquido",
            "Dim_Lojas.REGIAO_CHILLI","Dim_Produtos.GRUPO_CHILLI","Dim_Produtos.Grupo_Produto"]
    miss = [c for c in need if c not in df_hist.columns]
    if miss: raise ValueError(f"Faltando colunas no histórico: {miss}")

    # ------- Agregações em nível de REGIÃO -------
    # Contagem de lojas distintas por região e soma de receita líquida por região
    reg_lojas = df_hist.groupby("Dim_Lojas.REGIAO_CHILLI")["ID_Loja"].nunique()
    reg_receita = df_hist.groupby("Dim_Lojas.REGIAO_CHILLI")["Total_Preco_Liquido"].sum()
    # Junta agregações em uma única tabela por região e nomeia colunas
    agg_reg = (pd.concat([reg_receita, reg_lojas], axis=1)
                 .reset_index()
                 .rename(columns={"Total_Preco_Liquido":"regiao_receita_total",
                                  "ID_Loja":"regiao_num_lojas"}))
    # Receita média por loja na região (com proteção a divisão por zero)
    agg_reg["regiao_receita_por_loja"] = (
        agg_reg["regiao_receita_total"]/agg_reg["regiao_num_lojas"].replace(0,np.nan)
    ).fillna(0)

    # ------- Medianas por REGIÃO × GRUPO (para features; não definem alvo diretamente) -------
    grp_meds = (df_hist.groupby(["Dim_Lojas.REGIAO_CHILLI","Dim_Produtos.GRUPO_CHILLI"])
                       .agg(grupo_med_preco_varejo=("Total_Preco_Varejo","median"),
                            grupo_med_desconto=("Desconto","median"))
                       .reset_index())

    # ------- Volume por REGIÃO × GRUPO para definir Top-K e rótulo -------
    vol = (df_hist.groupby(["Dim_Lojas.REGIAO_CHILLI","Dim_Produtos.GRUPO_CHILLI"])["Total_Preco_Liquido"]
                 .sum().reset_index()
                 .rename(columns={"Total_Preco_Liquido":"grupo_regiao_receita"}))
    # Ranking dentro de cada região por receita do par (quanto menor o rank, maior a receita)
    vol["rank_na_regiao"] = (vol.groupby("Dim_Lojas.REGIAO_CHILLI")["grupo_regiao_receita"]
                                .rank(method="first", ascending=False))
    # Rótulo binário: Top-K na região recebe 1; demais recebem 0
    vol["Bom_Desempenho"] = (vol["rank_na_regiao"] <= top_k).astype(int)

    # ------- Montagem de features X e rótulo y (chave = REGIÃO × GRUPO) -------
    X = (vol[["Dim_Lojas.REGIAO_CHILLI","Dim_Produtos.GRUPO_CHILLI"]]
         .merge(agg_reg, on="Dim_Lojas.REGIAO_CHILLI", how="left")
         .merge(grp_meds, on=["Dim_Lojas.REGIAO_CHILLI","Dim_Produtos.GRUPO_CHILLI"], how="left"))
    y = vol["Bom_Desempenho"].astype(int)

    # ------- Catálogo por GRUPO (metadados úteis para cruzar em novas localidades) -------
    catalogo = (df_hist.groupby(["Dim_Produtos.GRUPO_CHILLI","Dim_Produtos.Grupo_Produto"])
                        .agg(Preco_Varejo_med=("Total_Preco_Varejo","median"))
                        .reset_index())

    # Retorna matriz de atributos, rótulo e agregados auxiliares
    return X, y, catalogo, agg_reg, grp_meds

# HISTÓRICO 
dfh = pd.read_csv(CAMINHO_DF_HIST)
# Remoção de colunas não utilizadas na construção de features/label
dfh = dfh.drop(columns=[
    "ID_Cliente","Dim_Cliente.Data_Nascimento","Dim_Cliente.Regiao_Cliente",
    "ID_Produto","Dim_Lojas.Nome_Emp","Dim_Lojas.Bairro_Emp","Dim_Lojas.Cidade_Emp",
    "Dim_Lojas.CANAL_VENDA","Dim_Lojas.Tipo_PDV","Dim_Lojas.Regiao"  # não usados
], errors="ignore")

# Construção da base supervisionada (X_all, y_all) e artefatos auxiliares
X_all, y_all, catalogo, agg_reg, grp_meds = build_train_table(dfh, TOP_K)
print(X_all.shape, y_all.mean().round(3))  # forma de X e taxa de positivos

# Visual 2.1 — Distribuição do alvo
vals = y_all.value_counts().sort_index()
plt.figure(figsize=(4,3))
plt.bar(vals.index.astype(str), vals.values)
plt.title("Distribuição do alvo (Bom_Desempenho)")
plt.xlabel("Classe"); plt.ylabel("Contagem")
plt.show()


### Definição do pipeline supervisionado (pré-processamento + RandomForest)

- **Objetivo:** encapsular, em um único `Pipeline`, o pré-processamento (imputação + codificação) e o classificador.
- **Entradas esperadas:** `X` com colunas categóricas (`Dim_Produtos.GRUPO_CHILLI`, `Dim_Lojas.REGIAO_CHILLI`) e numéricas (agregações regionais e medianas por par).
- **Pré-processamento:**
  - Categóricas: imputação por moda (`most_frequent`) e `OneHotEncoder` com `handle_unknown="ignore"` (compatível com categorias inéditas na inferência).
  - Numéricas: imputação por mediana (`median`).
  - Compatibilidade: seleção automática entre `sparse_output=False` (sklearn ≥ 1.2) e `sparse=False` (versões anteriores).
- **Modelo:** `RandomForestClassifier` com:
  - `class_weight="balanced"` para mitigar desbalanceamento do alvo,
  - `n_estimators=500`, `max_depth=12`, `min_samples_leaf=2` (viés-variância mais controlado),
  - `random_state` fixo e `n_jobs=-1` para paralelismo.
- **Saída:** objeto `Pipeline` pronto para `fit`, `predict_proba` e `predict`, mantendo consistência de transformações entre treino e inferência.


In [ ]:
def make_model(X):
    # Define dinamicamente as colunas categóricas presentes em X
    cat_cols = [c for c in ["Dim_Produtos.GRUPO_CHILLI","Dim_Lojas.REGIAO_CHILLI"] if c in X.columns]
    # Define as colunas numéricas esperadas (agregações regionais e medianas por par REGIÃO × GRUPO)
    num_cols = ["regiao_receita_total","regiao_num_lojas","regiao_receita_por_loja",
                "grupo_med_preco_varejo","grupo_med_desconto"]

    # OneHotEncoder: compatibilidade entre versões do sklearn (sparse_output vs sparse)
    if "sparse_output" in OneHotEncoder().get_params().keys():
        oh = OneHotEncoder(handle_unknown="ignore", sparse_output=False)  # sklearn mais recente
    else:
        oh = OneHotEncoder(handle_unknown="ignore", sparse=False)         # sklearn anterior

    # Pré-processamento por tipo de coluna: categóricas (imputação + one-hot) e numéricas (imputação)
    pre = ColumnTransformer(
        transformers=[
            ("cat", Pipeline([("imputer", SimpleImputer(strategy="most_frequent")), ("onehot", oh)]), cat_cols),
            ("num", Pipeline([("imputer", SimpleImputer(strategy="median"))]), num_cols),
        ],
        remainder="drop"  # descarta colunas não especificadas
    )

    # Classificador: RandomForest com pesos balanceados para lidar com desbalanceamento do alvo
    rf = RandomForestClassifier(
        n_estimators=500, max_depth=12, min_samples_leaf=2,
        class_weight="balanced", random_state=RANDOM_STATE, n_jobs=-1
    )

    # Retorna um Pipeline unificado (pré-processamento + modelo)
    return Pipeline([("pre", pre), ("clf", rf)])

# Instancia o pipeline final com base nas colunas de X_all (garante compatibilidade de features)
model = make_model(X_all)


### Validação e escolha de limiar (CV estratificada + hold-out com curvas)

- **Finalidade:** estimar desempenho médio via **Stratified K-Fold** e selecionar um **limiar de decisão** a partir de um **hold-out estratificado**.
- **Cross-validation (CV):**
  - Recria o `Pipeline` a cada dobra, ajusta em treino e avalia em teste.
  - Coleta **AP (PR-AUC)** e **ACC** usando limiar fixo 0,50 (diagnóstico inicial).
- **Hold-out (seleção de limiar):**
  - Separa 20% estratificado, treina, obtém **probabilidades** e calcula métricas.
  - Identifica o **limiar que maximiza F1** pela curva **Precision–Recall**.
  - Exibe gráficos: PR, **F1 × limiar**, ROC e **matriz de confusão** no limiar ótimo.
- **Saídas:**
  - Impressões de **AP/ACC médios** na CV.
  - Relatório de **hold-out** (classification report, matriz, ROC-AUC, PR-AUC).
  - **Limiar escolhido** (retornado para uso posterior).

In [ ]:
def crossval_scores(model, X, y):
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)  # CV estratificada e reprodutível
    aps, accs = [], []
    for tr, te in cv.split(X, y):
        m = Pipeline(model.steps)                 # recria o Pipeline para cada dobra (evita contaminação)
        m.fit(X.iloc[tr], y.iloc[tr])             # ajusta apenas com a parte de treino da dobra
        proba = m.predict_proba(X.iloc[te])[:,1]  # probabilidades da classe positiva no conjunto de teste
        pred  = (proba >= 0.5).astype(int)        # predição binária com limiar 0.50 (diagnóstico)
        aps.append(average_precision_score(y.iloc[te], proba))  # AP (PR-AUC) da dobra
        accs.append((pred == y.iloc[te]).mean())                 # acurácia da dobra
    print(f"\nCV (RF)  AP={np.mean(aps):.3f}  ACC={np.mean(accs):.3f}")  # médias das dobras

def choose_threshold_and_plots(model, X, y, limiar=None):
    if limiar is not None:
        return float(limiar)  # respeita limiar pré-definido (pula seleção automática)
    sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=RANDOM_STATE)  # hold-out estratificado 80/20
    tr, te = next(sss.split(X, y))
    model.fit(X.iloc[tr], y.iloc[tr])                     # ajuste no conjunto de treino do hold-out
    proba = model.predict_proba(X.iloc[te])[:,1]          # probabilidades no conjunto de teste
    yte   = y.iloc[te].values

    print("\n=== Hold-out (estratificado) — limiar 0.50 ===")
    pred050 = (proba >= 0.5).astype(int)                  # diagnóstico com limiar padrão 0.50
    print(classification_report(yte, pred050, digits=3))  # métricas detalhadas
    print("Matriz:\n", confusion_matrix(yte, pred050))
    print("ROC-AUC:", round(roc_auc_score(yte, proba), 3),
          " | PR-AUC:", round(average_precision_score(yte, proba), 3))

    # Curva PR e busca do limiar que maximiza F1
    prec, rec, thr = precision_recall_curve(yte, proba)   # pontos (precision, recall) por limiar
    f1s = 2*(prec*rec)/(prec+rec+1e-9)                    # F1 em cada ponto da curva
    idx = np.nanargmax(f1s)                                # índice do F1 máximo
    chosen = float(thr[idx]) if idx < len(thr) else 0.5    # limiar escolhido (fallback 0.5 se necessário)
    print(f"\n→ Limiar ótimo (F1 máx): {chosen:.3f}")

    # Visual 4.1 — Precision-Recall
    plt.figure(figsize=(4.2,3.2))
    plt.plot(rec, prec)                                    # curva PR
    plt.scatter(rec[idx], prec[idx])                       # marca o ponto de F1 máximo
    plt.title("Curva Precision–Recall (hold-out)")
    plt.xlabel("Recall"); plt.ylabel("Precision")
    plt.show()

    # Visual 4.2 — F1 vs Threshold
    plt.figure(figsize=(4.2,3.2))
    xs = thr if len(thr) else np.array([0.5])              # eixo x: limiares conhecidos (ou 0.5)
    ys = f1s[:-1] if len(thr) else np.array([0.0])         # eixo y: F1 correspondente (descarta último ponto PR)
    plt.plot(xs, ys)
    if len(thr):
        plt.axvline(chosen, linestyle="--")                # linha vertical no limiar escolhido
    plt.title("F1 × Limiar (hold-out)")
    plt.xlabel("Limiar"); plt.ylabel("F1")
    plt.show()

    # Visual 4.3 — ROC
    fpr, tpr, _ = roc_curve(yte, proba)
    plt.figure(figsize=(4.2,3.2))
    plt.plot(fpr, tpr)                                     # curva ROC
    plt.plot([0,1],[0,1])                                  # diagonal (classificador aleatório)
    plt.title("Curva ROC (hold-out)")
    plt.xlabel("FPR"); plt.ylabel("TPR")
    plt.show()

    # Visual 4.4 — Matriz de confusão (limiar ótimo)
    pred_opt = (proba >= chosen).astype(int)               # predição com o limiar selecionado
    cm = confusion_matrix(yte, pred_opt)
    plt.figure(figsize=(3.4,3))
    plt.imshow(cm)                                         # heatmap simples
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, cm[i,j], ha="center", va="center")  # valores nas células
    plt.title("Matriz de confusão (limiar ótimo)")
    plt.xticks([0,1], ["Neg","Pos"]); plt.yticks([0,1], ["Neg","Pos"])
    plt.xlabel("Predito"); plt.ylabel("Verdadeiro")
    plt.colorbar()
    plt.show()

    return chosen  # retorna o limiar escolhido para uso nas próximas etapas

# Execução
crossval_scores(model, X_all, y_all)                           # desempenho médio em CV (AP, ACC)
chosen_thr = choose_threshold_and_plots(model, X_all, y_all, LIMIAR)  # seleção do limiar final


### Treinamento final, persistência do modelo e análise de importância

- **Objetivo:** ajustar o modelo supervisionado em toda a base disponível, salvar o artefato treinado e interpretar globalmente a contribuição das variáveis.
- **Treinamento e salvamento:**
  - O modelo é ajustado em `X_all` e `y_all`.
  - O objeto resultante é salvo no arquivo definido em `ARQ_MODELO` (`.joblib`).
- **Importância por permutação:**
  - Mede a queda média de desempenho quando uma feature é embaralhada.
  - Calculada no próprio `X_all` (interpretação global do dataset).
  - Exibe tabela das variáveis mais relevantes e gráfico horizontal com o Top-10.
- **Validações rápidas:**
  - Conferir mensagem de salvamento do modelo.
  - Avaliar se as variáveis com maior importância fazem sentido para o negócio.


In [ ]:
# Treina final e salva
model.fit(X_all, y_all)                                    # ajuste final em toda a base disponível
dump(model, ARQ_MODELO.as_posix())                         # persistência do modelo em arquivo .joblib
print(f"\nModelo salvo em: {ARQ_MODELO.name}")

# Importância por permutação (no próprio conjunto X_all para interpretação global)
result = permutation_importance(
    model, X_all, y_all,
    n_repeats=10, random_state=RANDOM_STATE, n_jobs=-1     # 10 repetições, paralelizado
)
importances = pd.DataFrame({
    "feature": X_all.columns,
    "importance_mean": result.importances_mean,            # importância média
    "importance_std": result.importances_std               # desvio padrão das repetições
}).sort_values("importance_mean", ascending=False)

display(importances.head(15))                              # tabela das 15 variáveis mais relevantes

# Visual 5.1 — Top-10 importâncias
top = importances.head(10).iloc[::-1]                      # seleciona top-10 e inverte para plotar de baixo para cima
plt.figure(figsize=(5,3.8))
plt.barh(top["feature"], top["importance_mean"])           # gráfico horizontal
plt.title("Importância por Permutação — Top 10")
plt.xlabel("Δ métrica (mean)")
plt.ylabel("Feature")
plt.tight_layout()
plt.show()


### Novas localidades e geração do ranking de categorias

- **Objetivo:** aplicar o modelo treinado às novas localidades sugeridas, calcular a probabilidade de sucesso e gerar um ranking das categorias mais promissoras.
- **Processo:**
  - Leitura e normalização dos dados de novas localidades (`novas_raw`).
  - Padronização de nomes de colunas (`estado`, `regiao_chilli`) para alinhar com o histórico.
  - Geração de candidatos por produto: combinação cruzada de cada nova localidade com o catálogo de grupos de produtos.
  - Enriquecimento dos candidatos com agregados regionais (`agg_reg`) e medianas regionais por grupo (`grp_meds`).
  - Seleção das mesmas features utilizadas no treino (`feat_cols`) para manter consistência.
  - Predição da **probabilidade de sucesso** e classificação binária com base no limiar escolhido (`chosen_thr`).
  - Ordenação dos resultados e seleção do **Top-N** por localidade.
- **Saídas:**
  - DataFrame `saida` com ranking das categorias por nova localidade, contendo escopo, cluster, grupo, preço mediano, probabilidade e previsão binária.


In [ ]:
# -------------------- NOVAS LOCALIZAÇÕES + RANKING --------------------
novas_raw = read_csv_flex(ARQ_NOVAS)  # leitura das novas localidades sugeridas
print("\nColunas nas novas localizações:", list(novas_raw.columns))

# Normaliza nomes de colunas para compatibilidade com o histórico
rename_map = {}
for c in novas_raw.columns:
    cl = c.strip().lower()
    if cl == "estado":
        rename_map[c] = "Dim_Lojas.Estado_Emp"
    elif cl == "regiao_chilli":
        rename_map[c] = "Dim_Lojas.REGIAO_CHILLI"
novas = novas_raw.rename(columns=rename_map).copy()
# Garante a presença das colunas identificadoras mínimas
for c in ["escopo","cluster_id"]:
    if c not in novas.columns: novas[c] = "NA"

# Criação de candidatos: cada nova localidade × cada grupo de produto
base = novas.rename(columns={"Dim_Lojas.REGIAO_CHILLI":"REG_TMP"}, errors="ignore").copy()
cand = base.merge(
    catalogo[["Dim_Produtos.GRUPO_CHILLI","Dim_Produtos.Grupo_Produto","Preco_Varejo_med"]],
    how="cross"  # produto cartesiano
)

# Anexa agregados regionais e medianas de grupo (vindos do histórico)
cand = cand.merge(agg_reg, left_on="REG_TMP", right_on="Dim_Lojas.REGIAO_CHILLI", how="left")
cand = cand.merge(grp_meds, left_on=["REG_TMP","Dim_Produtos.GRUPO_CHILLI"],
                  right_on=["Dim_Lojas.REGIAO_CHILLI","Dim_Produtos.GRUPO_CHILLI"], how="left",
                  suffixes=("","_dup"))

# Ajusta coluna de região para manter consistência com treino
cand["Dim_Lojas.REGIAO_CHILLI"] = cand["REG_TMP"]

# Seleciona as mesmas features usadas no treinamento
feat_cols = ["Dim_Lojas.REGIAO_CHILLI","Dim_Produtos.GRUPO_CHILLI",
             "regiao_receita_total","regiao_num_lojas","regiao_receita_por_loja",
             "grupo_med_preco_varejo","grupo_med_desconto"]
cand_feat = cand[feat_cols].copy()

# Predição da probabilidade e classe (binária) para cada candidato
proba = model.predict_proba(cand_feat)[:,1]
cand["Probabilidade_Sucesso"] = proba
cand["Previsao"] = (proba >= chosen_thr).astype(int)
cand["Probabilidade_pct"] = (proba*100).round(1)  # em porcentagem arredondada

# Ranking Top-N por localidade
id_cols = ["escopo","cluster_id"]
rank = cand.sort_values(id_cols + ["Probabilidade_Sucesso"], ascending=[True, True, False]).reset_index(drop=True)
top_n = rank.groupby(id_cols, group_keys=False).head(TOP_N)

# Seleciona colunas de saída
cols_out = (id_cols + ["Dim_Lojas.REGIAO_CHILLI","Dim_Produtos.GRUPO_CHILLI","Dim_Produtos.Grupo_Produto",
                       "Preco_Varejo_med","Probabilidade_pct","Previsao"])
saida = top_n[[c for c in cols_out if c in top_n.columns]].copy()


### Exibição e exportação do ranking por localidade

- **Objetivo:** apresentar os resultados finais de ranking das categorias por nova localidade e salvar em arquivo `.csv`.
- **Processo:**
  - Impressão no console: para cada par `escopo × cluster_id`, mostra as categorias ranqueadas (sem repetir colunas de identificação).
  - Visualização: gráfico de barras com as probabilidades previstas das **Top-N categorias** do primeiro local.
  - Exportação: grava o ranking consolidado em `ranking_grupo_chilli_por_local_simplificado.csv` para uso externo.
- **Validações rápidas:**
  - Conferir se cada localidade aparece com seu respectivo Top-N.
  - Avaliar se o gráfico de barras reflete corretamente a ordem das probabilidades.
  - Confirmar a criação do arquivo `.csv` no diretório de saída.


In [ ]:
print("\n==================== RANKING TOP-N POR LOCAL ====================")
# Itera sobre cada local (escopo × cluster_id) e imprime tabela formatada
for (esc, clu), df_loc in saida.groupby(["escopo","cluster_id"]):
    print(f"\n>>> Local: escopo={esc} | cluster_id={clu}  (Top-{TOP_N})")
    # imprime apenas colunas relevantes (sem escopo e cluster_id)
    print(df_loc.drop(columns=["escopo","cluster_id"]).to_string(index=False))

# Visual 7.1 — Para o primeiro local, barras das Top-N probabilidades
first_key = next(iter(saida.groupby(["escopo","cluster_id"]).groups.keys()))  # pega o primeiro local
ex = saida.set_index(["escopo","cluster_id"]).loc[first_key].reset_index(drop=True)
plt.figure(figsize=(6,3.5))
plt.bar(range(len(ex)), ex["Probabilidade_pct"])                             # barras = probabilidades (%)
plt.xticks(range(len(ex)), ex["Dim_Produtos.GRUPO_CHILLI"], rotation=45, ha="right")
plt.ylabel("Probabilidade (%)")
plt.title(f"Top-{TOP_N} — {first_key}")
plt.tight_layout()
plt.show()

# Exportação do resultado consolidado
ARQ_OUT = "../../../database/dataset gerado/ranking_grupo_chilli_por_local_simplificado.csv"
saida.to_csv(ARQ_OUT, index=False, sep=";")
print(f"\nArquivo exportado: {ARQ_OUT}")


### Checagem de consistência: nulos e cobertura de regiões

- **Objetivo:** verificar a qualidade e consistência dos dados antes de finalizar a etapa supervisionada.
- **Processo:**
  - **Nulos nas features de treino:** contabiliza valores ausentes em `X_all` e lista as 10 variáveis mais afetadas.
  - **Cobertura de regiões:** compara as regiões presentes nas novas localidades (`novas`) com aquelas do histórico agregado (`agg_reg`).
- **Critérios de atenção:**
  - Se existirem muitas features com nulos, pode ser necessário revisar imputação ou fontes de dados.
  - Se alguma região nova não tiver correspondência no histórico, o modelo pode não gerar predições confiáveis para ela.
- **Saídas:**
  - Relatório textual com top-10 variáveis mais afetadas por nulos.
  - Aviso indicando se existem regiões novas sem histórico ou confirmação de cobertura completa.

In [ ]:
# Checa nulos nas features de treino
nulls = X_all.isna().sum().sort_values(ascending=False)
print("Nulos nas features (top 10):")
print(nulls.head(10).to_string())  # exibe as 10 variáveis com mais valores ausentes

# Verifica cobertura de regiões nas novas vs. histórico
if "Dim_Lojas.REGIAO_CHILLI" in novas.columns:
    regs_novas = set(novas["Dim_Lojas.REGIAO_CHILLI"].dropna().unique())  # regiões presentes nas novas localidades
    regs_hist  = set(agg_reg["Dim_Lojas.REGIAO_CHILLI"].unique())         # regiões existentes no histórico agregado
    faltando = regs_novas - regs_hist
    if faltando:
        print("\n⚠ Regiões nas novas SEM histórico agregado:", faltando)
    else:
        print("\n✔ Todas as regiões das novas têm histórico agregado.")


### Visualização do ranking exportado (`ARQ_OUT`)

- **Objetivo:** inspecionar rapidamente o arquivo de ranking exportado e destacar padrões úteis para decisão.
- **Conteúdo:**
  - Leitura do CSV exportado (`ARQ_OUT`) e **amostra** das linhas.
  - **Histograma** das probabilidades previstas (visão geral de confiança).
  - **Gráfico de barras** das Top-N categorias do primeiro local (leitura rápida do mix recomendado).
  - **Heatmap simples** (localidade × grupo) com a probabilidade (%) — facilita comparar locais/categorias.
  - **Tabela ranqueada** por local (escopo, cluster) com as principais colunas.
- **Observações:**
  - Os gráficos usam apenas `matplotlib` e não definem cores específicas.
  - Caso o arquivo não exista, a célula informa o problema sem interromper o notebook.


In [ ]:
# =================== Visualização do ranking exportado ===================
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 1) Leitura e amostra
if not os.path.exists(ARQ_OUT):
    print(f"⚠ Arquivo não encontrado: {ARQ_OUT}. Execute a etapa de exportação antes desta célula.")
else:
    df_rank = pd.read_csv(ARQ_OUT, sep=";")
    print(f"✔ Ranking carregado: {ARQ_OUT} | shape={df_rank.shape}")
    display(df_rank.head(10))

    # Confere colunas essenciais
    required_cols = {"escopo","cluster_id","Dim_Produtos.GRUPO_CHILLI","Probabilidade_pct"}
    missing = required_cols - set(df_rank.columns)
    if missing:
        print(f"⚠ Colunas ausentes para visualizações completas: {missing}")

    # 2) Histograma das probabilidades (visão global)
    if "Probabilidade_pct" in df_rank.columns:
        plt.figure(figsize=(5.5,3.6))
        plt.hist(df_rank["Probabilidade_pct"].dropna(), bins=20)
        plt.xlabel("Probabilidade prevista (%)")
        plt.ylabel("Frequência")
        plt.title("Distribuição de Probabilidades (Ranking Exportado)")
        plt.tight_layout()
        plt.show()

    # 3) Barras das Top-N do primeiro local (escopo × cluster)
    if {"escopo","cluster_id","Dim_Produtos.GRUPO_CHILLI","Probabilidade_pct"} <= set(df_rank.columns):
        first_key = next(iter(df_rank.groupby(["escopo","cluster_id"]).groups.keys()))
        amostra = (df_rank.set_index(["escopo","cluster_id"])
                           .loc[first_key]
                           .reset_index(drop=True))
        plt.figure(figsize=(6,3.6))
        plt.bar(range(len(amostra)), amostra["Probabilidade_pct"])
        plt.xticks(range(len(amostra)), amostra["Dim_Produtos.GRUPO_CHILLI"], rotation=45, ha="right")
        plt.ylabel("Probabilidade (%)")
        plt.title(f"Top-N — {first_key}")
        plt.tight_layout()
        plt.show()

    # 4) Heatmap simples (escopo × grupo) de Probabilidade_pct
    if {"escopo","cluster_id","Dim_Produtos.GRUPO_CHILLI","Probabilidade_pct"} <= set(df_rank.columns):
        # Seleciona um subconjunto de locais para manter o gráfico legível (ex.: até 8 locais)
        keys = list(df_rank.groupby(["escopo","cluster_id"]).groups.keys())[:8]
        sub = (df_rank.set_index(["escopo","cluster_id"])
                       .loc[keys]
                       .reset_index())
        # Pivot: linhas = local (escopo|cluster), colunas = grupo, valores = prob %
        sub["local"] = sub["escopo"].astype(str) + " | " + sub["cluster_id"].astype(str)
        piv = sub.pivot_table(index="local", columns="Dim_Produtos.GRUPO_CHILLI",
                              values="Probabilidade_pct", aggfunc="max")
        # Preenche ausências com 0 para exibição
        M = piv.fillna(0).values
        plt.figure(figsize=(max(6, 0.35*M.shape[1]+3), max(3.5, 0.35*M.shape[0]+2)))
        plt.imshow(M, aspect="auto")
        plt.xticks(range(piv.shape[1]), piv.columns, rotation=45, ha="right")
        plt.yticks(range(piv.shape[0]), piv.index)
        plt.colorbar(label="Probabilidade (%)")
        plt.title("Heatmap — Probabilidade por Local × Grupo (amostra)")
        plt.tight_layout()
        plt.show()

    # 5) Tabela ranqueada por local (impressão legível)
    cols_print = [c for c in [
        "escopo","cluster_id","Dim_Lojas.REGIAO_CHILLI",
        "Dim_Produtos.GRUPO_CHILLI","Dim_Produtos.Grupo_Produto",
        "Preco_Varejo_med","Probabilidade_pct","Previsao"
    ] if c in df_rank.columns]

    print("\n==================== RANKING (amostra tabular ordenada) ====================")
    for (esc, clu), df_loc in df_rank.groupby(["escopo","cluster_id"]):
        print(f"\n>>> Local: escopo={esc} | cluster_id={clu}")
        df_show = df_loc.sort_values("Probabilidade_pct", ascending=False)[cols_print].copy()
        # Limita a impressão a 12 linhas por local para manter a leitura rápida
        print(df_show.head(12).to_string(index=False))
